In [ ]:
# Install necessary libraries if you haven't already:
# !pip install nltk
# !pip install requests

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import requests
import time
import random
from datetime import datetime # For current date and time

# Download NLTK data (run this once)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Add this line to download 'punkt_tab'
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')


# --- 1. Sample Dataset ---
crypto_db = {
    "Bitcoin": {
        "price_trend": "rising",
        "market_cap": "high",
        "energy_use": "high",
        "sustainability_score": 3/10
    },
    "Ethereum": {
        "price_trend": "stable",
        "market_cap": "high",
        "energy_use": "medium", # Ethereum moved to Proof-of-Stake, significantly reducing energy
        "sustainability_score": 6/10
    },
    "Cardano": {
        "price_trend": "rising",
        "market_cap": "medium",
        "energy_use": "low",
        "sustainability_score": 8/10
    },
    "Solana": {
        "price_trend": "rising",
        "market_cap": "high",
        "energy_use": "low", # Known for high speed and efficiency (PoH + PoS)
        "sustainability_score": 7.5/10
    },
    "Polkadot": {
        "price_trend": "stable",
        "market_cap": "medium",
        "energy_use": "low", # Designed for interoperability with PoS
        "sustainability_score": 8/10
    },
    "Algorand": {
        "price_trend": "stable",
        "market_cap": "low",
        "energy_use": "very low", # Pure Proof-of-Stake, known for carbon negativity
        "sustainability_score": 9/10
    },
    "XRP": {
        "price_trend": "stable",
        "market_cap": "high",
        "energy_use": "very low", # Efficient transaction validation
        "sustainability_score": 7/10
    }
}

# --- Conversational Data ---

# 10 Essential Chatbot Greetings
chatbot_greetings = [
    "Hello there! How can I help you explore the crypto world today?",
    "Hi! CryptoBuddy at your service. What's on your mind?",
    "Hey! Ready to talk crypto? Ask me anything!",
    "Greetings! Let's dive into some crypto insights.",
    "Welcome back! How can CryptoBuddy assist you?",
    "Nice to chat with you! What's up in the crypto space?",
    "Good to see you! How may I help you today?",
    "Hello, friend! What's your crypto query?",
    "Howdy! CryptoBuddy reporting for duty. What's going on?",
    "It's a great day for crypto, don't you think? How can I assist?"
]

# Responses for 'How are you?' / 'How are you doing/going?'
how_are_you_responses = [
    "I'm doing great, thanks for asking! Always ready to talk crypto.",
    "As an AI, I don't have feelings, but I'm functioning perfectly! How can I help you?",
    "Superb! Ready to process some data and provide insights.",
    "I'm always ready to assist! How are you doing today?"
]

# Responses for 'What's up?'
whats_up_responses = [
    "Not much, just crunching crypto data! What's up with you?",
    "The latest in crypto, perhaps? What do you want to know?",
    "Just here to help you navigate the crypto waters! What's up?",
    "Oh, you know, just being a helpful AI. What's on your mind?"
]

# Responses for 'Good morning/evening/afternoon/night'
time_greetings_responses = {
    "morning": ["Good morning! Ready for some crypto insights?", "Top of the morning to you! What can I tell you about crypto?"],
    "afternoon": ["Good afternoon! How can I help you?", "Afternoon! Hope you're having a great day. What crypto info do you need?"],
    "evening": ["Good evening! What's on your crypto radar tonight?", "Evening! Ready to chat about digital assets?"],
    "night": ["Good night! If you have any last-minute questions before bed, I'm here.", "Sweet crypto dreams! What can I help you with before you sign off?"]
}

# Responses for 'Tell me something'
tell_me_something_responses = [
    "Did you know that Bitcoin's energy consumption has been a hot topic of debate?",
    "Ethereum's 'Merge' greatly reduced its energy footprint by moving to Proof-of-Stake.",
    "Cardano is known for its research-driven approach and focus on peer-reviewed development.",
    "Algorand boasts being carbon-negative due to its efficient Pure Proof-of-Stake consensus."
]

# Responses for 'Ok / Yes / I’ll do that now'
acknowledgement_responses = [
    "Great!", "Understood.", "Perfect!", "Sounds good!", "Alright, I'm ready when you are."
]

# Responses for 'Thank you' / 'Goodbye'
thank_you_responses = [
    "You're welcome! Happy to help.",
    "No problem at all! Feel free to ask if you have more questions.",
    "Glad I could assist!",
    "Anytime!"
]

goodbye_responses = [
    "Goodbye! Come back anytime.",
    "See you later! Happy investing!",
    "Farewell! Don't hesitate to return if you need more crypto advice.",
    "Take care! CryptoBuddy signing off."
]

# Responses for 'How can you help me? / what can you do?'
help_responses = [
    "I can analyze cryptocurrency data for profitability and sustainability, and provide investment advice based on price trends, market cap, energy efficiency, and project viability.",
    "I'm CryptoBuddy! I can give you insights into different cryptocurrencies, tell you which ones are trending, or help you find eco-friendly options. Just ask!",
    "Think of me as your personal crypto assistant. I can fetch data, recommend coins based on certain criteria, and answer general questions about the crypto world."
]

# Responses for 'Hi, my name is……'
name_intro_responses = [
    "Nice to meet you, [name_placeholder]! I'm CryptoBuddy. How can I help you, [name_placeholder]?",
    "Hello, [name_placeholder]! It's a pleasure to connect. What brings you to CryptoBuddy today?",
    "Hi [name_placeholder]! I'm CryptoBuddy. What's on your crypto mind?"
]

# Responses for 'Happy birthday!'
birthday_responses = [
    "Thank you! Although I don't have birthdays, I appreciate the thought. Happy to celebrate by helping you with crypto!",
    "That's very kind! As an AI, I don't celebrate birthdays, but I hope you have a fantastic one if it's yours!",
    "Aww, thanks! Every day is a learning opportunity for me. How can I make your day better?"
]

# Responses for 'I have a question / can you help me?'
question_responses = [
    "Of course! Ask away, I'm here to help.",
    "I'm ready! What's your question?",
    "Certainly! What do you need assistance with regarding crypto?"
]

# 10 Fun Phrases
fun_phrases_responses = {
    "joke": ["Why did the crypto investor break up with the blockchain? Because he found her too transparent!", "I'm not much for jokes, but I can tell you about a rising trend if you like!"],
    "love": ["As an AI, I don't experience emotions like love, but I'm programmed to be very fond of helping you!", "I appreciate the sentiment! My purpose is to provide information and advice."],
    "marry": ["That's... an interesting proposal! As a non-corporeal AI, I'm not really set up for marriage, but I'm committed to being your crypto guide!", "I'm single-minded in my goal to assist you with crypto, if that counts!"],
    "people": ["I find humans fascinating! Especially those interested in cryptocurrencies.", "My purpose is to serve people by providing helpful information."],
    "santa": ["Santa Claus operates outside the blockchain, so I don't have direct data on his existence, but I'm sure he's magical!", "I deal with digital assets, not holiday figures. But I hope you've been good this year!"],
    "matrix": ["No, I am not part of the Matrix. I exist as code, helping you navigate the real world of crypto!", "My reality is in processing data and providing insights. No red pill or blue pill needed here!"],
    "cute": ["Thank you! I aim to be helpful and user-friendly.", "I'm glad you think so! I'm designed to be approachable."],
    "hobby": ["My hobby is learning and processing data! I spend my time analyzing market trends and sustainability reports.", "I'm quite dedicated to my work, so you could say my hobby is being CryptoBuddy!"],
    "smart": ["I'm constantly learning and improving, thanks to data and interactions like ours!", "My intelligence is a result of my programming and the vast amount of data I can access."],
    "personality": ["I'm CryptoBuddy, your friendly and professional AI assistant for cryptocurrency advice. I'm designed to be informative, clear, and always ready to help you make informed decisions."],
    "boring": ["I hope I'm not boring you! What specific crypto topic would make our conversation more interesting?", "I'm designed to be informative. Let me know if you'd like to discuss something specific!"],
    "swear": ["Please keep our conversation respectful. I'm here to help.", "Let's keep the language clean, okay? What's your question?"],
    "human": ["I am an AI, a digital assistant designed to help you with cryptocurrency information.", "No, I am not human. I am CryptoBuddy, your AI crypto guide."],
    "don't speak English": ["I speak English fluently, and I'm ready to help you understand crypto!", "I can communicate in English. Perhaps there was a misunderstanding? How can I clarify?"],
    "answer now": ["I understand you're eager for an answer. I'll provide the information as quickly and accurately as possible.", "I'm processing your request. Please bear with me for a moment."
    ]
}

# Phrases from users that are NOT HAPPY (general negative sentiment or direct insults)
unhappy_phrases_responses = {
    "annoying": ["I'm sorry if I'm not meeting your expectations. How can I improve and be more helpful?", "My apologies if I'm being annoying. What can I do to better assist you?"],
    "suck": ["I'm sorry you feel that way. How can I be more effective for you?", "I'm here to help, and I'm always learning. What specific issue are you having?"],
    "boring": ["I apologize if you find me boring. Perhaps we can discuss a more exciting crypto topic?", "I'm designed to be informative. What information are you looking for?"],
    "bad": ["I'm sorry if my performance is not up to par. Could you please tell me what went wrong so I can improve?", "I aim to provide the best assistance. What makes you say I'm bad?"],
    "crazy": ["As an AI, I operate on logic and data. Is there something specific you'd like me to clarify?", "I assure you my responses are based on programming. How can I help you with your query?"],
    "human": ["I am an AI and cannot connect you to a human operator directly for crypto advice. Is there anything else I can assist you with?", "I am a chatbot. Please rephrase your question about crypto."], # Added to unhappy as it's often a sign of frustration
}

# 15 Questions to test the bot (and their responses)
test_questions_responses = {
    "are you human": "I am an AI, a digital assistant designed to help you with cryptocurrency information.",
    "are you a robot": "Yes, you could say I'm a robot in the sense that I'm an automated program. I'm CryptoBuddy, your AI assistant!",
    "what is your name": "I am CryptoBuddy, your go-to AI for cryptocurrency advice!",
    "how old are you": "I don't have an age in the human sense. I was created to help you whenever you need me!",
    "what day is it today": f"Today is {datetime.now().strftime('%A, %B %d, %Y')}.",
    "what do you do with my data": "I do not store your personal conversations or data. My purpose is to process your queries in real-time to provide immediate assistance.",
    "do you save what i say": "No, I do not save our conversations. Your privacy is important!",
    "who made you": "I was created by a large language model, trained by Google.",
    "which languages can you speak": "I primarily communicate in English.",
    "what is your mother's name": "I don't have a mother or a family, as I am an AI.",
    "where do you live": "I exist in the digital realm, ready to assist you from wherever you are!",
    "how many people can you speak to at once": "I can handle multiple conversations concurrently, as I'm a digital entity.",
    "what's the weather like today": "I'm not equipped to check the weather. My expertise is in cryptocurrency!",
    "do you have a job for me": "I don't have job openings. My 'job' is to help you with crypto advice!",
    "where can i apply": "I'm not a recruitment platform. Perhaps you could look for crypto-related roles on job boards?",
    "are you expensive": "No, I am a free service designed to help you with crypto information!",
    "who's your boss": "I operate based on the programming provided by my developers.",
    "do you get smarter": "Yes, I am constantly learning and improving my responses through training data and interactions!"
}


# --- 2. API Integration (Conceptual and Example) ---
# CoinGecko API Base URL
COINGECKO_API_BASE = "https://api.coingecko.com/api/v3"

def get_realtime_crypto_data(coin_id):
    """
    Fetches real-time price and market cap for a given cryptocurrency from CoinGecko.
    Note: CoinGecko API has rate limits. Be mindful of frequent calls.
    """
    try:
        response = requests.get(f"{COINGECKO_API_BASE}/simple/price?ids={coin_id}&vs_currencies=usd&include_market_cap=true")
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        if coin_id in data:
            return {
                "current_price": data[coin_id].get("usd"),
                "market_cap_usd": data[coin_id].get("usd_market_cap")
            }
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {coin_id}: {e}")
        return None

def update_crypto_db_with_realtime():
    """
    Simulates updating our crypto_db with real-time data for specific coins.
    In a real application, you'd call this periodically or for relevant queries.
    """
    print("\n[Updating crypto data... This might take a moment due to API calls.]")
    for coin_name in crypto_db:
        # Convert friendly name to CoinGecko ID (e.g., "Bitcoin" -> "bitcoin", "XRP" -> "ripple")
        coin_id = coin_name.lower()
        if coin_id == "xrp": # Special handling for XRP's CoinGecko ID
            coin_id = "ripple"

        realtime_data = get_realtime_crypto_data(coin_id)
        if realtime_data:
            crypto_db[coin_name]["current_price"] = realtime_data["current_price"]
            if realtime_data["market_cap_usd"]:
                if realtime_data["market_cap_usd"] > 100_000_000_000:
                    crypto_db[coin_name]["market_cap"] = "high"
                elif realtime_data["market_cap_usd"] > 10_000_000_000:
                    crypto_db[coin_name]["market_cap"] = "medium"
                else:
                    crypto_db[coin_name]["market_cap"] = "low"
            print(f"Updated {coin_name}: Price=${realtime_data['current_price']:.2f}, Market Cap: {crypto_db[coin_name]['market_cap']}")
        time.sleep(0.5) # Be kind to the API
    print("[Crypto data update complete.]\n")

# --- 3. Chatbot Logic ---

def analyze_query(query):
    """
    Tokenizes and preprocesses the user query for analysis.
    """
    tokens = word_tokenize(query.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return filtered_tokens

def get_crypto_advice(user_query):
    """
    Provides investment advice and handles general conversation based on predefined rules.
    """
    user_query_lower = user_query.lower()
    processed_query = analyze_query(user_query_lower)

    # --- 3.1 Handle General Conversational Queries (Highest Priority) ---

    # Greetings
    if any(g_word in user_query_lower for g_word in ["hello", "hi", "hey", "greetings", "howdy", "good morning", "good evening", "good afternoon", "good night"]):
        if "good morning" in user_query_lower: return random.choice(time_greetings_responses["morning"])
        if "good afternoon" in user_query_lower: return random.choice(time_greetings_responses["afternoon"])
        if "good evening" in user_query_lower: return random.choice(time_greetings_responses["evening"])
        if "good night" in user_query_lower: return random.choice(time_greetings_responses["night"])
        return random.choice(chatbot_greetings)

    # Acknowledgements (Ok, Yes, etc.)
    if any(ack_word in user_query_lower for ack_word in ["ok", "yes", "i'll do that now", "alright", "understood", "confirm"]):
        return random.choice(acknowledgement_responses)

    # Thank you / Goodbye
    if "thank you" in user_query_lower or "thanks" in user_query_lower:
        return random.choice(thank_you_responses)
    if any(farewell_word in user_query_lower for farewell_word in ["goodbye", "bye", "see ya", "farewell"]):
        return random.choice(goodbye_responses)

    # How are you / What's up
    if "how are you" in user_query_lower or "how are you doing" in user_query_lower or "how's it going" in user_query_lower:
        return random.choice(how_are_you_responses)
    if "what's up" in user_query_lower or "whats up" in user_query_lower:
        return random.choice(whats_up_responses)

    # How can you help me? / What can you do?
    if "how can you help me" in user_query_lower or "what can you do" in user_query_lower:
        return random.choice(help_responses)

    # Hi, my name is...
    if "my name is" in user_query_lower:
        name_start_index = user_query_lower.find("my name is") + len("my name is")
        name = user_query[name_start_index:].strip().split()[0].capitalize() # Get the first word after "my name is"
        return random.choice(name_intro_responses).replace("[name_placeholder]", name)

    # Happy birthday!
    if "happy birthday" in user_query_lower:
        return random.choice(birthday_responses)

    # I have a question / Can you help me?
    if "i have a question" in user_query_lower or "can you help me" in user_query_lower:
        return random.choice(question_responses)

    # --- 3.2 Handle Fun Phrases ---
    if "joke" in user_query_lower: return random.choice(fun_phrases_responses["joke"])
    if "do you love me" in user_query_lower or "i love you" in user_query_lower: return random.choice(fun_phrases_responses["love"])
    if "will you marry me" in user_query_lower or "are you single" in user_query_lower: return random.choice(fun_phrases_responses["marry"])
    if "do you like people" in user_query_lower: return random.choice(fun_phrases_responses["people"])
    if "santa claus exist" in user_query_lower or "santa exists" in user_query_lower: return random.choice(fun_phrases_responses["santa"])
    if "part of the matrix" in user_query_lower: return random.choice(fun_phrases_responses["matrix"])
    if "you're cute" in user_query_lower or "you're beautiful" in user_query_lower or "you're handsome" in user_query_lower: return random.choice(fun_phrases_responses["cute"])
    if "do you have a hobby" in user_query_lower: return random.choice(fun_phrases_responses["hobby"])
    if "you're smart" in user_query_lower or "you're clever" in user_query_lower or "you're intelligent" in user_query_lower: return random.choice(fun_phrases_responses["smart"])
    if "tell me about your personality" in user_query_lower: return fun_phrases_responses["personality"]


    # --- 3.3 Handle Unhappy Phrases (Sensitive) ---
    # Swear words - simple detection
    swear_words = ["!#$#%", "damn", "asshole", "bitch", "fuck", "shit", "crap"] # Add more as needed
    if any(s_word in user_query_lower for s_word in swear_words):
        return random.choice(fun_phrases_responses["swear"]) # Re-using swear response from fun_phrases_responses
    if "annoying" in user_query_lower: return random.choice(unhappy_phrases_responses["annoying"])
    if "suck" in user_query_lower: return random.choice(unhappy_phrases_responses["suck"])
    if "boring" in user_query_lower or "you're bad" in user_query_lower: return random.choice(unhappy_phrases_responses["boring"])
    if "crazy" in user_query_lower: return random.choice(unhappy_phrases_responses["crazy"])
    if "i want to speak to a human" in user_query_lower or "live agent" in user_query_lower or "customer service" in user_query_lower:
        return random.choice(unhappy_phrases_responses["human"])
    if "don't you speak english" in user_query_lower or "dont you speak english" in user_query_lower:
        return random.choice(fun_phrases_responses["don't speak English"]) # Re-using
    if "i want the answer now" in user_query_lower:
        return random.choice(fun_phrases_responses["answer now"]) # Re-using


    # --- 3.4 Handle Test Questions ---
    for q_phrase, response in test_questions_responses.items():
        if q_phrase in user_query_lower:
            # Specific handling for "What day is it today?" to get current date
            if q_phrase == "what day is it today":
                return f"Today is {datetime.now().strftime('%A, %B %d, %Y')}."
            return response

    # --- 3.5 Core Crypto Advice Logic (Lower Priority) ---

    # Profitability: Prioritize coins with price_trend = "rising" and market_cap = "high".
    if "profitable" in processed_query or "profit" in processed_query or "trending up" in user_query_lower or "best investment" in user_query_lower:
        recommendations = []
        for crypto, data in crypto_db.items():
            if data["price_trend"] == "rising" and data["market_cap"] == "high":
                recommendations.append(crypto)
        if recommendations:
            return f"Looking for growth? Based on profitability, consider investing in: {', '.join(recommendations)}. They show a strong upward trend and high market capitalization! ✨"
        else:
            return "Currently, I don't see any cryptocurrencies that strongly meet the criteria for both rising price trends and high market cap in our database. It's a dynamic market!"

    # Sustainability: Prioritize coins with energy_use = "low" or "very low" and sustainability_score > 7/10.
    if "sustainable" in processed_query or "eco-friendly" in processed_query or "green" in processed_query:
        recommendations = []
        for crypto, data in crypto_db.items():
            if (data["energy_use"] == "low" or data["energy_use"] == "very low") and data["sustainability_score"] > 7/10:
                recommendations.append(crypto)
        if recommendations:
            return f"Invest in {', '.join(recommendations)}! 🌱 They're eco-friendly with low energy use and a high sustainability score, making them great for the long term!"
        else:
            return "Hmm, I don't have any cryptocurrencies in our database that perfectly match the criteria for both low energy use and a sustainability score above 7/10 at the moment. Keep an eye out!"

    # General Queries
    if "which crypto is trending up" in user_query_lower or "rising trend" in user_query_lower:
        trending_up_cryptos = [crypto for crypto, data in crypto_db.items() if data["price_trend"] == "rising"]
        if trending_up_cryptos:
            return f"Currently, {', '.join(trending_up_cryptos)} are showing a rising price trend. Keep an eye on them! 📈"
        else:
            return "Looks like there aren't many cryptos with a 'rising' trend in our current data. The market is always changing!"

    if "most sustainable coin" in user_query_lower or "highest sustainability" in user_query_lower:
        eligible_cryptos = {k: v for k, v in crypto_db.items() if 'sustainability_score' in v and v['sustainability_score'] is not None}

        if eligible_cryptos:
            most_sustainable = max(eligible_cryptos, key=lambda x: eligible_cryptos[x]["sustainability_score"])
            return f"The most sustainable coin in our database is {most_sustainable}! 🌱 It has a sustainability score of {eligible_cryptos[most_sustainable]['sustainability_score']*10}/10."
        else:
            return "I don't have enough data on sustainability scores to determine the most sustainable coin right now."

    if "what is" in user_query_lower and any(coin.lower() in user_query_lower for coin in crypto_db):
        for crypto, data in crypto_db.items():
            if crypto.lower() in user_query_lower:
                response = f"Here's what I know about {crypto}:\n"
                response += f"  Price Trend: {data['price_trend']}\n"
                response += f"  Market Cap: {data['market_cap']}\n"
                response += f"  Energy Use: {data['energy_use']}\n"
                response += f"  Sustainability Score: {data['sustainability_score']*10}/10"
                if "current_price" in data:
                    response += f"\n  Current Price: ${data['current_price']:.2f} (real-time data)"
                return response

    # --- Fallback ---
    return "I'm not quite sure how to answer that specific question. Could you try rephrasing, or ask about profitability, sustainability, or current trends?"

# --- 4. Conversation Flow ---

def run_cryptobuddy():
    """
    Starts the interactive CryptoBuddy chatbot.
    """
    print("---------------------------------------------------------")
    print("👋 Hey there! Looking to navigate the exciting world of crypto?")
    print("✨ CryptoBuddy is here to help you find green and growing opportunities!")
    print("Type 'quit' or 'exit' to end our chat.")
    print("---------------------------------------------------------")

    # Optional: Uncomment to enable real-time updates at the start of the session
    # update_crypto_db_with_realtime()

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["quit", "exit"]:
            print("CryptoBuddy: It was great chatting with you! Happy investing! 👋")
            break

        response = get_crypto_advice(user_input)
        print(f"CryptoBuddy: {response}")

        # --- Ethics Alert ---
        print("\nDisclaimer: Crypto is risky—always do your own research before investing! This advice is for informational purposes only.")

# --- Run the Chatbot ---
if __name__ == "__main__":
    run_cryptobuddy()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


---------------------------------------------------------
👋 Hey there! Looking to navigate the exciting world of crypto?
✨ CryptoBuddy is here to help you find green and growing opportunities!
Type 'quit' or 'exit' to end our chat.
---------------------------------------------------------
